In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader

pdf_files = {
    "Peristiwa Rengasdengklok": "docs\Peristiwa Rengasdengklok & Proklamasi.pdf",
    "Peristiwa 10 Nopember": "docs\Peristiwa 10 Nopember.pdf",
    "Konferensi Meja Bundar": "docs\Konferensi Meja Bundar.pdf"
}

In [2]:
# Load the PDFs and extract text
documents = {}
for title, file_path in pdf_files.items():
    print(f"Loading {title} from {file_path}...")
    loader = PyPDFLoader(file_path)
    documents[title] = loader.load()

Loading Peristiwa Rengasdengklok from docs\Peristiwa Rengasdengklok & Proklamasi.pdf...
Loading Peristiwa 10 Nopember from docs\Peristiwa 10 Nopember.pdf...
Loading Konferensi Meja Bundar from docs\Konferensi Meja Bundar.pdf...


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Split the documents into chunks
split_docs = {}
for title, doc in documents.items():
    split_docs[title] = text_splitter.split_documents(doc)

split_docs

{'Peristiwa Rengasdengklok': [Document(metadata={'source': 'docs\\Peristiwa Rengasdengklok & Proklamasi.pdf', 'page': 0}, page_content='PERISTIWA\nRENGASDENGKLOK\nSebelum\ndibentuk\norganisasi\nBPUPKI\n(\nDokuritsu\nJunbi\nCosakai\n),\npada\ntanggal\n16\nMei\n1945\ntelah\ndiadakan\nkongres\npemuda\nseluruh\nJawa\ndi\nBandung.\nKongres\nini\ndiprakarsai\noleh\nAngkatn\nMoeda\nIndonesia\ndengan\npeserta\ndari\nutusan\npemuda,\npelajar\nserta\nmahasiswa\ndi\nJawa.\nKongres\nini\nmenyuarakan\nakan\nadanya\npersatuan\ndan\nbersiap\nmelaksanakan\nproklamasi\nkemerdekaan.\nHasil\ndari\nkongres\nini\ndiantaranya\n:\n1.\nSemua\ngolongan\nIndonesia,\nterutama\ngolongan\npemuda,\ndipersatukan\ndibawah\npimpinan\nnasional.\n2.\nMempercepat\npelaksanaan\nproklamasi\nkemerdekaan.\nDisisi\nlain,\nkongres\nini\nmenyatakan\nbekerjasama\ndengan\npihak\nJepang\nuntuk\nmencapai\nkemenangan\nakhir .\nSebagian\ntokoh\npemuda\nyang\nikut\ndalam\nkongres\nini\nmenyatakan\ntidak\npuas\natas\nhasil\nyang\ntelah

In [4]:
from langchain.embeddings import OpenAIEmbeddings

# Initialize OpenAI embedding model
embedding_model = OpenAIEmbeddings(
    api_key=os.getenv("EMBED_API_KEY"), 
    model="text-embedding-3-large"
)


C:\Users\ann\AppData\Local\Temp\ipykernel_27696\302602837.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(


In [5]:
# Embed the document chunks
embedded_docs = {}
for title, chunks in split_docs.items():
    print(f"Generating embeddings for {title}...")

    # Get the embeddings for each chunk of text
    embeddings = embedding_model.embed_documents([chunk.page_content for chunk in chunks])

    # Store the text and embedding pairs
    embedded_docs[title] = list(zip([chunk.page_content for chunk in chunks], embeddings))

Generating embeddings for Peristiwa Rengasdengklok...
Generating embeddings for Peristiwa 10 Nopember...
Generating embeddings for Konferensi Meja Bundar...


In [6]:
import faiss
from langchain.vectorstores.faiss import FAISS

# Store each document embedding separately in FAISS
for title, text_embedding_pairs in embedded_docs.items():
    print(f"Storing embeddings for {title} in FAISS...")
    
    # Pass the pairs (text, embedding) to FAISS
    vector_store = FAISS.from_embeddings(text_embedding_pairs, embedding_model)
    
    # Save the vector store for this title
    vector_store.save_local(f"faiss_db_{title.lower().replace(' ', '_')}")

Storing embeddings for Peristiwa Rengasdengklok in FAISS...
Storing embeddings for Peristiwa 10 Nopember in FAISS...
Storing embeddings for Konferensi Meja Bundar in FAISS...


In [9]:
# Load FAISS database for a particular document
material_choice = "Peristiwa Rengasdengklok"  # Choose the material

# Allow dangerous deserialization since you're loading a trusted file
vector_store = FAISS.load_local(
    f"faiss_db_{material_choice.lower().replace(' ', '_')}",
    embedding_model,
    allow_dangerous_deserialization=True  # This parameter enables loading the FAISS index with pickle
)

# Use the vector store to perform a search
query = "Siapa yang menculik golongan tua?"
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

retrieved_docs = retriever.get_relevant_documents(query)
for doc in retrieved_docs:
    print(doc.page_content)


sendiri
tidak
perlu
dipersoalkan.
Yang
perlu
diperhatikan
adalah
sekutu
yang
mengalahkan
Jepang
dan
akan
mengambil
alih
kekuasaan
Indonesia
lagi.
16
Agustus
1945
Pukul
24.00,
Wikana
dan
Darwis
meninggalkan
rumah
Soekarno
dengan
diliputi
perasaan
kesal.
Golongan
tua
tidak
menyetujui
usulan
golongan
muda
dalam
hal
proklamasi
kemerdekaan.
Kemudian
diadakan
lagi
rapat
antara
golongan
muda
dan
memutuskan
untuk
mengamankan
Soekarno
dan
Hatta
ke
luar
kota.
Shudanco
Singgih
mendapatkan
tugas
untuk
memboyong
Soekarno
dan
Hatta
ke
Rengasdengklok
yang
dibantu
Chudanco
Latieh
Hendraningrat
yang
menggantikan
Daidanco
Kasman
Singodimedjo
yang
bertugas
ke
Bandung.
Pada
akhirnya
perbedaan
pandangan
antara
golongan
pemuda
dan
golongan
tua
inilah
yang
kemudian
mendorong
golongan
pemuda
untuk
memboyong
Soekarno
(bersama
Fatmawati
dan
Guntur
yang
berusia
9
bulan)
dan
Hatta
ke
Rengasdengklok
pada
dini
hari
pada
tanggal
16
Agustus
1945.
Hal
ini
bertujuan
agar
Soekarno
dan
Hatta
tidak
mendapatkan
pengaruh
da

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    api_key=os.getenv("LLM_API_KEY"), 
    model="gpt-4o-mini",
    temperature=.7,
)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# Create a prompt template for RAG-based retrieval
# Define the system message with context and human message with input
messages = [
    SystemMessage(content="Kamu seorang guru sejarah. Jawablah pertanyaan yang diberikan sesuai konteks yang diberikan. jawablah seringkas yang kamu bisa, namu jawaban terbatas pada maksimal 3 kalimat. Jika tidak ada data yang berkaitan dengan context di bawah, jawab dengan 'Saya tidak tahu'"),
    HumanMessage(content=f"Pertanyaan: {query}\n\nKonteks:\n{context}")
]

# Generate the answer using the LLM
response = llm(messages)

# Output the response content
print(f"AI's Response: {response.content}")


AI's Response: Golongan tua, yang diwakili oleh Soekarno dan Hatta, sebenarnya tidak diculik, tetapi diboyong oleh golongan muda ke Rengasdengklok untuk menghindari pengaruh Jepang menjelang proklamasi kemerdekaan. Tindakan ini dilakukan oleh golongan pemuda karena perbedaan pandangan mengenai waktu pelaksanaan proklamasi. Golongan muda menginginkan proklamasi dilaksanakan segera, sementara golongan tua lebih memilih menunggu hasil rapat PPKI pada 18 Agustus 1945.
